<a id='section_id1'></a>
# BOM Weather

### A project evaluating the Persistence Model against the AUS Bureau of Meteorology.
We often get a 7 day weather forecast but don't often go back to see how accurate the predictions were for 7 days ago.   
This project looks to explore how accurate the weather forecast is according to a what is known as the Persistence Model.
The Persistence Model hypothesis for the weather domain is that "The weather tomorrow will be the same as today",   
or in mathematical terms Weather(t+1) = Weather(t), (t being today, or chosen point in time).

The BOM forecasts are known to be very accurate for t = 1,2 and 3 days into the future, so this project will evaluate the whole 7 day forecast.

The persistence model, also called the naïve predictor, is often used as a reference as it is a good ground estimation of other algorithms,and often used as a reference for determining the skill factor of a competing forecast model.   

The second part of this project will try to predict the weather, using the persistence model as a feature, giving some adjustment to standard forcasting models Facebook prophet and Random Forrest trained on historical data.

All data is available for free via the BOM website and examples are stored in the repository.

[Click here for EDA notebook locally](eda.ipynb#section_id1)   
[Click here for EDA notebook GitHub](https://github.com/bfgdigital/BOM_Weather/blob/main/notebooks/eda.ipynb)

**Imports**

In [13]:
import pandas as pd
import numpy as np
import requests
from matplotlib import pyplot as plt
import datetime as dt

Establish a date for today

In [14]:
today = dt.date.today()
todaystr = today.strftime("%Y-%m-%d")

yesterday = dt.date.today() - pd.DateOffset(days=1)
yesterdaystr = yesterday.strftime("%Y-%m-%d")

tomorrow = dt.date.today() + pd.DateOffset(days=1)
tomorrowstr = tomorrow.strftime("%Y-%m-%d")

day_after_tomorrow = dt.date.today() + pd.DateOffset(days=2)
day_after_tomorrowstr = day_after_tomorrow.strftime("%Y-%m-%d")

print(f'Yesterdays\'s date was: {yesterday.date()}')
print(f'Today\'s date is: {today}')
print(f'Tomorrows\'s date is: {tomorrow.date()}')
print(f'Day after that is: {day_after_tomorrow.date()}')

Yesterdays's date was: 2020-10-30
Today's date is: 2020-10-31
Tomorrows's date is: 2020-11-01
Day after that is: 2020-11-02


### DATA Sources 
- Data Comes From: ftp://ftp.bom.gov.au/anon/gen/fwo/
- Melbourne Forecast File: ftp://ftp.bom.gov.au/anon/gen/fwo/IDV10450.xml

The url for the BOM API is:
https://api.weather.bom.gov.au/v1/locations/r1r143/forecasts/daily   
Be Aware that the update is adjusted every 10mins.

In [15]:
locations = {
'LakeEildon' : 'https://api.weather.bom.gov.au/v1/locations/r1ru3qn/forecasts/daily',
'FallsCreek' : 'https://api.weather.bom.gov.au/v1/locations/r32wr3e/forecasts/daily',
'Mildura' : 'https://api.weather.bom.gov.au/v1/locations/r1vjdbz/forecasts/daily',
'Corryong' : 'https://api.weather.bom.gov.au/v1/locations/r394jdk/forecasts/daily',
'WilsonsProm' : 'https://api.weather.bom.gov.au/v1/locations/r3046k1/forecasts/daily',
'Melbourne' : 'https://api.weather.bom.gov.au/v1/locations/r1r143/forecasts/daily',
} # Melbourne last so it will be used for this iteration of the project.

**Save full forecast for future reference and development**

In [16]:
for name,url in locations.items():
    response = requests.get(url)
    weather_dict = response.json() # format as json
    latest_forecast = weather_dict
    api_forecast = pd.DataFrame(latest_forecast['data'])
    api_forecast.index=api_forecast['date']
    filename = '../data/forecast_records/forecast_' + str(name) + '_' + todaystr + '.csv'
    api_forecast.to_csv(filename) # backup file.

Create a call backup to prevent constant pinging of API during development.

In [5]:
def call_backup():
    latest_forecast = weather_dict.copy()
    return latest_forecast

call_backup();

**Check Forecast date is correct**   
Establish current dates using datetime

API values change depending on the time of day.   
We need to handle the dates with a forecast status.

In [6]:
issue_time = pd.to_datetime(latest_forecast['metadata']['issue_time']).date()
print('\n',f'Forecast was issued on: {issue_time}')


 Forecast was issued on: 2020-10-29


In [7]:
forecast_status = 0
if issue_time == today:
    print('API forecast row 0 is for tomorrow')
    forecast_status = 1
elif issue_time == yesterday:
    print('API forecast row 0 is for today')
else:
    print('Check API data.')
    forecast_status = 2

API forecast row 0 is for tomorrow


In [8]:
lf = pd.DataFrame(latest_forecast['data'])
lf['date'] = pd.to_datetime(lf['date']).dt.date

if forecast_status == 1: # We want to bump the date column forwards 1 day.
    lf[['date']] = lf[['date']] + pd.DateOffset(days=1) # add 1 day
    print(f"Forecast date index moved forwards 1 day.")
else:
    forecast_status == 0 # We want to bump the date column forwards 1 day.
    print(f"Dates are correct, no need to change index")

lf.index = lf['date']
print('\n',f"Todays Max: {lf.index[0]} {list(lf['temp_max'][:1])}, Forecasts: {list(lf['temp_max'][1:])}")

Forecast date index moved forwards 1 day.

 Todays Max: 2020-10-29 00:00:00 [22], Forecasts: [20, 18, 19, 26, 28, 21, 16]


**Load Temps File**

In [9]:
temps = pd.read_csv('../data/temps.csv',infer_datetime_format=True,index_col=0)

# Reset location.
def reset_location():
    temps = pd.read_csv('../data/temps.csv',infer_datetime_format=True,index_col=0)
    return temps

Merge with existing forecast file

In [10]:
# TBC
# call_backup();
number_of_forecasts = len(latest_forecast['data'])

### Summary of new data
Latest observations taken from BOM

In [11]:
class color:
   BOLD = '\033[1m'
   END = '\033[0m'

print(f"New forecasts:	{number_of_forecasts}")
print(f"Starting on:	{lf['date'][0]}")
print(f"Ending on:	{lf['date'][-1]}")
print(f"Today's Temp:		{lf['temp_max'][0]}")
print(f"Tomorrow's Temp:	{lf['temp_max'][1]}","\n")
print(color.BOLD + f"Here's today's forecast: \n{lf['extended_text'][0]}" + color.END)

New forecasts:	8
Starting on:	2020-10-29 00:00:00
Ending on:	2020-11-05 00:00:00
Today's Temp:		22
Tomorrow's Temp:	20 

Here's today's forecast: 
Cloudy. Medium (50%) chance of showers, becoming less likely late this evening. The chance of thunderstorms early this evening, mainly about northern and eastern suburbs. Winds south to southwesterly 15 to 25 km/h becoming light in the evening.


### Append new data to existing data

In [12]:
roll_days = ['location','today+0','today+1','today+2','today+3','today+4','today+5','today+6']
location = [name]
temperatures = list(lf['temp_max'][0:])
string = location + temperatures
print(f'New Row: {string}, with {len(string)-1}/7 days')
new_forecast_row = pd.Series(string, index=roll_days, name=today)

if today.strftime("%Y-%m-%d") in temps.index:
    print(f'File not saved. Date already exists in index.')
else:
    temps = temps.append(new_forecast_row)
    file_name = '../data/temps_' + today.strftime("%Y-%m-%d") + '.csv'
    temps.to_csv(file_name) # backup file.
    temps.to_csv('../data/temps.csv')
    print('File Saved')

New Row: ['Melbourne', 22, 20, 18, 19, 26, 28, 21, 16], with 8/7 days


ValueError: Length of passed values is 9, index implies 8.

In [ ]:
# temps.to_csv(file_name) # backup file.
# temps.to_csv('temps.csv')

Review New Data

In [ ]:
temps.info()
temps['location'].unique()

### Choose location

In [ ]:
reset_location()
temps = temps[temps['location'] == 'Melbourne'].drop(['location'], axis=1)

# PART 2: Plotting Data
- Load and plot weather forecasts.

In [ ]:
# Seaborn lineplot, takes care of most settings.
import seaborn as sns

In [ ]:
def plot_data(data,title):
    sns.set(rc={'figure.figsize':(12,5)})
    ax = sns.lineplot(data=data.T,legend=None, dashes=False,) # transpose df and dashes beyond 6 cols thows errors.
    ax.set_title(title, loc='center', fontsize=18)
    return plt.show();

In [ ]:
plot_data(temps,"All Forecasts")

The above chart shows a pattern, but isn't clealy interpretable.

In [ ]:
def heat_map(data,title):
    fig, ax = plt.subplots(figsize = (10,10))
    ax = sns.heatmap(data, annot=True, center=True, cmap = 'coolwarm',cbar_kws={'label': 'Degrees Celcius'})
    ax.set_title(title, loc='center', fontsize=18)
    ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14, rotation=30)
    ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14, rotation=0)
    ax.figure.axes[-1].yaxis.label.set_size(14)
    ax.figure.axes[0].yaxis.label.set_size(14)
    ax.figure.axes[0].xaxis.label.set_size(14)
    return plt.show();

In [ ]:
heat_map(temps,"7 Day Forecasts From BOM (Decending to the left)")

**Evaluation: This chart should be read top to bottom, right to left.**       
As the forecast date approaches (decending), the the colour change to the left reflects stregnthening and weaking of systems and the daily adjustment of forecasts.

# PART 3: Evaluate Forecast Accuracy
Check forecast temperature for each day against the recorded temperature for each day.

In [ ]:
basic_dates = temps.index # Keep non-datetime index list for charts. (avoid 00:00:00 ending)
temps.index = pd.to_datetime(temps.index) # make index a datetime.

def highlight_diags(data):
    '''Highlight Forecast Lines'''
    attr1 = 'background-color: lightgreen'
    attr2 = 'background-color: salmon'
    attr3 = 'background-color: lightblue'
    
    df_style = data.replace(data.values, '')
    np.fill_diagonal(np.flipud(df_style), attr1)
    np.fill_diagonal(np.flipud(df_style)[2:], attr2)
    np.fill_diagonal(np.flipud(df_style)[4:], attr3)
    return df_style

temps.style.apply(highlight_diags, axis=None)

The date being forecast shifts down and to the left as the date approaches.   
Note, the highlighting does not appear on GitHub. Please view file here. [Please view file here](https://github.com/bfgdigital/BOM_Weather/tree/main/assets/forecast_highlighting.png)


In [ ]:
# Acuracy Mechanism: Compare forecast to actual Temp.
fac = pd.DataFrame()
counter = list(range(len(temps)))
columns = list(temps.columns)

for i in counter:
    # 7 day forecast inc today, so len can't exceed 7
    if i < 7:
        window = i 
        j = i
    else: 
        window = 6
        j = 6
    
    # Start date at most recent row
    actual_date = temps.index[-1] # start with the last day
    window_date = actual_date - pd.DateOffset(days=window) # Number of days in the past can't be more than those forecast
    row_0 = temps.index[0] # We want to end when window date is equal to row_0.
    
    temps_list = [] # temporary holder of weeeks values.
    while window_date >= row_0:
        true_temp = int(temps.loc[actual_date][0]) # True temperature recorded on day
        predicted_temp = int(temps.loc[window_date][window]) # data predicted on value of window
        difference =  true_temp - predicted_temp
        # loop 
        actual_date -= pd.DateOffset(days=1) # take off 1 day.
        window_date -= pd.DateOffset(days=1) # take off 1 day.
        # append
        temps_list.append(difference)    
    # Add list to df as series    
    fac[columns[j]] = pd.Series(temps_list[::-1]) # Add list backwards.
        
fac.index = basic_dates
fac, temps

In [ ]:
heat_map(fac,"Forecast Variation (0 = 100% Accurate)")

**Evaluation: This chart should be read top to bottom, right to left.**       
Forecasts are generally accurate out to 3 days, with some error developing from 4 days on wards.  What is noteable is the error surrounding weather events. The largest amount of error occurs during the change in weather, as a front approaches for eg, while stable weather appears to be more easily predicted.

# PART 4: Build Persistence Model
- Build models for  Weather(t+i) = Weather(t)

In [ ]:
# Persistance Mechanism subtract each max temp from the one before.
pmodel  = pd.Series([today - yesterday for today,yesterday in zip(temps['today+0'],temps['today+0'][1:])],index=temps.index[:len(temps.index)-1])
pmodel

This shows that the persistance model we have is correct as each value in the list is the sum of the sequential temperature value subtracted. This gives us a baseline for how accurate "Tomorrow's weather will be the same as today" was for the days observed to date.

# PART 5: Compare the two models
- Compare Persistance to the forecasts provided by the BOM

In [ ]:
persistence = pd.DataFrame()
persistence['Persistence Accuracy'] = pmodel.values
for i in range(1,7):
    persistence[str(i)+' Day Forecast'] = pd.Series(fac['today+'+str(i)].values)
persistence.index = basic_dates[:len(basic_dates)-1]
persistence

In [ ]:
heat_map(persistence,"Persistence (far left) vs Forecast")

It appears that the persistance model may be a good benchmark for forecasts greater than 4 days away.   
To summarise, if you were to wager that the weather on a date 5 days from now, would be the closer to that of the day before rather than what the BOM has forecast, you would likely win.   

This suggests the feasibility of using the persistence model as a weighted feature in forecasts.

# PART 6: Can we use the Persistence model as a feature?
This section wil attemtp to use historical data from the BOM with common forecasting models Facebook Prophet and Random Forest to see if combined with a Persisence model, we can forecast more accurately than the BOM.

## New Forecast
Using cleaned BOM data from EDA.   
[Click here for EDA notebook locally](eda.ipynb#section_id1)   
[Click here for EDA notebook GitHub](https://github.com/bfgdigital/BOM_Weather/blob/main/notebooks/eda.ipynb)

In [ ]:
weather = pd.read_csv('../data/weather.csv',infer_datetime_format=True,index_col=0)
weather.tail()

### Build forecast model
Facebook Prophet is a popular and simple timeseries forecsting tool that is particularly effective with seasonal data.   
["fbprophet" documentation can be found here](https://facebook.github.io/prophet/docs/quick_start.html)

In [ ]:
from fbprophet import Prophet

In [ ]:
# Create a timeseries
start_date = weather['max_temp_c'].index[0] # First data point
end_date = weather['max_temp_c'].index[-1] # Last data point.
print(f'Range begins {start_date} and ends {end_date}.')

Prophet needs Date and values columns (ds and y)

In [ ]:
pdf = pd.DataFrame(None)
pdf['ds'] = weather.index # Create our date column
pdf['y'] = weather['max_temp_c'].values
pdf

### Observations.
- Data goes all the way back to 2013 recorded daily.   

Because we have timeseries data we can resample and smooth the data using a Savgol filter to adjust for big temperature changes that occur suddenly that BOM forecasts appear to struggle with.

In [ ]:
pdf.index = pd.to_datetime(pdf['ds'])
pdf = pdf.resample('6h').mean().bfill() # Resample dates to 6h intervals and back fill.
pdf['ds'] = pdf.index
print(pdf)
plot_data(pdf['y'],"Resampled To 6h Intervals") # plot y
pdf = pdf.reset_index(drop=True) # Perfer to have sequential index for smoothing.

### Smoth Values
- SavGol filter is excellent for smoothing a timeseries data.

This generates smoother curves for the Prophet model to use for interpreting seasonality. Effectively softening outliers.

In [ ]:
from scipy.signal import savgol_filter
# Smooth with SavGol filter
pdf_sg = pdf
savgol = [savgol_filter(pdf_sg[i], 5, 3, mode='constant') for i in pdf_sg] # modes: 'mirror', 'constant', 'nearest', 'wrap' or 'interp'
savgol = pd.DataFrame(savgol, columns=pd.to_datetime(pdf_sg.index), index=pdf_sg.columns)
savgol = savgol.T # Needed a flip
pdf['y'] = savgol['y']
plot_data(pdf['y'],"Smoothed Values") # plot y

## Use Facebook Prophet to forecast tomorrow's temperature
Facebook Prophet has the ability to detect cycles which may help make more accurate forecasts   
as more data is collected.

In [ ]:
model = Prophet(weekly_seasonality=False)
model.fit(pdf)
future = model.make_future_dataframe(periods = 2) # forecast 2 days beyond available data.
forecast = model.predict(future)
figure = model.plot(forecast, xlabel = 'Date', ylabel = 'Temperature')
print('\n',f"The forecast for tomorrow suggested by Facebook Prophet is: {float(forecast['yhat'][len(forecast)-1]):.2f}",'\n')
prophet_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(2)  # show the dataframe.
prophet_forecast.reset_index(drop=True,inplace=True)
prophet_forecast

Prophet model appears to offer reasonable values.   
Next to check the identified components.

In [ ]:
model.plot_components(forecast);

This looks promising, the Yearly trends look accurate the daily trend introduced by the Savgol filter looks to represent overnight temperature fluctuations of around 8º and the trend is based on the latest observations.

## Forecast Max Temp with Random Forrest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
y = weather['max_temp_c']
X = weather.drop(['max_temp_c'], axis=1)

regr = RandomForestRegressor(max_depth=100, random_state=42)
regr.fit(X, y)

## Fetch today's rainfall, and uv_index from our API forecast

**Assign feature predictors**   
BOM only supplies 2 days worth of UV predictors.

In [ ]:
q_rf = 0 # assuming quality data
q_maxt = 0 # assuming quality data
q_mint = 0 # assuming quality data
max_temps_2days = []
min_temps_2days = []
tomorrow_2days = []
rain_2days = []
uv_2days = []
for i in range(1,3):
    max_temps_2days.append(lf['temp_max'][i])
    min_temps_2days.append(lf['temp_min'][i])
    try:
        rain_2days.append(float(lf['rain'][i]['amount']['max'])) # Using Max instead of Max-Min
    except:
        rain_2days.append(0) # Nan value when no rain.
    uv_2days.append(lf['uv'][i]['max_index'])

rf_features = pd.DataFrame([min_temps_2days, max_temps_2days, rain_2days, uv_2days],columns=['today+1','today+2'],index=['MinTemp','MaxTemp','Rain','UV'])
rf_features

In [ ]:
randomforest_forecast = []
for i in rf_features:
    [predicted_mt] = regr.predict([[rf_features.loc['Rain'][i], q_rf, rf_features.loc['MinTemp'][i], q_mint, q_maxt, rf_features.loc['UV'][i]]]) # sbracket both sides to remove from list.
    score = regr.score(X, y) # R^2 (coefficient of determination) regression score function.
    randomforest_forecast.append(predicted_mt)
    print(f"The temperature forecast by the BOM tomorrow is {rf_features.loc['MaxTemp'][i]} and by our model is {predicted_mt:.2f}, with {score:.2f}% accuracy")

<div class="alert alert-info"><strong>NOTE:</strong> This output also appears to return an acceptable output, however this model is still using future BOM foreast data for rain and uv, which technically means this model is somewhat cheating.   
All the same it is still an interesting forecast.   
    
This model could be used with the persistence model of using today's recorded values for rain and uv features, however I will use the persistence model as part of the final model in the next step and don't want to over-weight yesterday's temperatures.</div>

# Putting all the pieces together
Build a new forecast and add it to existing forecasts.

In [ ]:
bens_vs_bom = pd.read_csv('../data/ben_vs_bom.csv',infer_datetime_format=True,index_col=0)

In [ ]:
tomorrow = latest_forecast['data'][1]['date'][:10] # We want it as a string, not a datetime.
dayafter = latest_forecast['data'][2]['date'][:10] # We want it as a string, not a datetime.

if dayafter in bens_vs_bom.index:
    print(f'Date is already in index, plese try later.')
else:
    # Day 1 Forecasts
    persistence_forecast_1 = int(temps[::-1]['today+0'][0]) # Today's temp
    prophet_forecast_1 = float(prophet_forecast['yhat'][0]) # Prophet temp
    randomforest_forecast_1 = float(randomforest_forecast[0]) # Random Forest temp
    bens_forecast_1 = (persistence_forecast_1 + prophet_forecast_1 + randomforest_forecast_1)/3 # Average of all three guesses.
    BOM_forecast_1 = rf_features.loc['MaxTemp'][0] # BOM forecast
    
    # Day 1 Forecasts
    persistence_forecast_2 = bens_forecast_1 # Yesterday's forecast temp
    prophet_forecast_2 = float(prophet_forecast['yhat'][1]) # Prophet temp
    randomforest_forecast_2 = float(randomforest_forecast[1]) # Random Forest temp
    bens_forecast_2 = (persistence_forecast_2 + prophet_forecast_2 + randomforest_forecast_2)/3 # Average of all three guesses.
    BOM_forecast_2 = rf_features.loc['MaxTemp'][1] # BOM forecast

    forecasts_1 = [persistence_forecast_1, prophet_forecast_1, randomforest_forecast_1, bens_forecast_1, BOM_forecast_1]
    forecasts_2 = [persistence_forecast_2, prophet_forecast_2, randomforest_forecast_2, bens_forecast_2, BOM_forecast_2]
    
    final_set = pd.DataFrame(None,columns=bens_vs_bom.columns)
    final_set.loc[0] = pd.Series(forecasts_1, index=bens_vs_bom.columns)
    final_set.loc[1] = pd.Series(forecasts_2, index=bens_vs_bom.columns)
    final_set.index = [tomorrow,dayafter]
    
    if dayafter in bens_vs_bom.index:
        print(f'File not saved. Date already exists in index, see you tomorrow.')
    else:
        bvb_filename = '../data/forecast_records/bvb_archive_' + todaystr + '.csv'
        bens_vs_bom.to_csv(bvb_filename) # backup file before dropping last value.
        bens_vs_bom.drop(bens_vs_bom.tail(1).index,inplace=True) # drop last row and update. (same as BOM model updates, dropping last value)
        bens_vs_bom = bens_vs_bom.append(final_set)
        bens_vs_bom.to_csv('../data/ben_vs_bom.csv') # backup file.
        print('File Saved, Thanks for coming.')

Forecast is built.

### So to wrap up, what do our models forecast the weather will be tomorrow?

In [ ]:
bens_vs_bom.style.set_properties(**{'background-color': 'yellow'}, subset=['Bens Best Guess'])

## Conclusion: 

<div class="alert alert-info"><strong>The forecast is close, but get your weather forecast from the BOM.</strong> </div>

The End.